In [1]:
import pandas as pd
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt

from numpy import mean
from numpy import std
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

In [2]:
# link to drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
def initialise(data):
  data.index = data["date"]
  data.index = pd.to_datetime(data.index)
  data.drop(columns=["date"], inplace=True)
  return data

# data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_mapped.csv'))
# data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_mapped.csv'))
# data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_mapped.csv'))
# data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_mapped_pp_nosic.csv'))
# data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_mapped_pp_nosic.csv'))
# data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_mapped_pp_nosic.csv'))

data_level = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_level_mapped_selection10.csv'))
data_slope = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_slope_mapped_selection10.csv'))
data_curve = initialise(pd.read_csv('/content/drive/MyDrive/Msc Thesis/code/data/Low resolution/data_curve_mapped_selection10.csv'))

data_level.head()

,secid,-80,-50,-20,level,slope,curve,permno,beta,betasq,...,cash,cinvest,roaq,roavol,ms,baspread,maxret,retvol,std_dolvol,std_turn
date,,,,,,,,,,,,,,,,,,,,,
1996-01-31,100862,0.307688,0.315627,0.358095,0.315627,0.042468,0.017264,54594,-0.502126,-0.886445,...,-0.868511,0.071798,0.396279,-0.953113,0.00,-0.689049,-0.662673,-0.619551,0.327234,-0.936622
1996-01-31,100871,0.705411,0.615432,0.651160,0.615432,0.035728,0.062854,50906,0.551365,0.193703,...,-0.738289,0.080067,0.538981,-0.900059,0.75,-0.112575,-0.390130,-0.223204,-0.342296,-0.748948
1996-01-31,100892,0.270090,0.216768,0.216508,0.216768,-0.000260,0.026531,57904,-0.536730,-0.902575,...,-0.958006,0.066337,0.367446,-0.998951,0.25,-0.786859,-0.947305,-0.877289,-0.536138,-0.955821
1996-01-31,100896,0.430796,0.436291,0.429517,0.436291,-0.006774,-0.006135,77520,0.349363,-0.101800,...,-0.968644,0.060739,0.459243,-0.859083,-0.25,-0.747736,-0.649524,-0.703492,-0.331142,-0.818216
1996-01-31,100903,0.262405,0.252683,0.373839,0.252683,0.121156,0.065439,80303,-0.186662,-0.682642,...,-0.584473,0.077826,0.605842,-0.875346,-0.50,-0.807682,-0.825541,-0.737079,-0.400503,-0.866238


In [5]:
data_level.tail()

,secid,-80,-50,-20,level,slope,curve,permno,beta,betasq,...,cash,cinvest,roaq,roavol,ms,baspread,maxret,retvol,std_dolvol,std_turn
date,,,,,,,,,,,,,,,,,,,,,
2021-12-31,151720,0.240322,0.258739,0.306341,0.258739,0.047602,0.014592,13132,-0.326826,-0.776237,...,-0.733093,0.225891,0.437399,-0.935844,0.00,-0.672456,-0.770928,-0.678560,-0.823942,-0.623090
2021-12-31,154591,0.167430,0.195268,0.243910,0.195268,0.048642,0.010402,13126,-0.573228,-0.910805,...,-0.733093,0.225891,0.437399,-0.935844,0.00,-0.799744,-0.883931,-0.796695,-0.779975,-0.987036
2021-12-31,151413,0.468498,0.476839,0.620803,0.476839,0.143964,0.067811,13142,0.686772,0.416767,...,-0.935171,0.224645,0.314647,-0.841808,-0.75,-0.446479,-0.097971,-0.302741,-0.690051,-0.951428
2021-12-31,151427,0.476894,0.447218,0.473760,0.447218,0.026542,0.028109,13169,-0.619074,-0.929135,...,-0.488685,0.225741,0.400845,-0.883241,0.25,-0.331974,-0.462362,-0.463095,-0.810640,-0.950397
2021-12-31,151076,0.212500,0.227045,0.281990,0.227045,0.054945,0.020200,13092,-0.089233,-0.588903,...,-0.733093,0.225891,0.437399,-0.935844,0.00,-0.771983,-0.908291,-0.761298,-0.464524,-0.821171


In [ ]:
def samplesplitting(data,train,validate,oos,i):
    train_data    = data.iloc[0 : train + i*oos, :]
    validate_data = data.iloc[train + i*oos : train + validate + i*oos , :]
    total_train   = data.iloc[0 : train + validate + i*oos, :]
    oos_data      = data.iloc[train + validate + i*oos : train + validate + (i+1)*oos, :]
    pred_data     = data.iloc[train + validate : train + validate + (i+1)*oos]

    print('train=%d, validate=%d, oos=%d' % (len(train_data), len(validate_data), len(oos_data)))
    return train_data, validate_data, oos_data, total_train, pred_data

def split_xy(data, kind):
  data_x = data.iloc[:,8:]
  data_y = data[kind]
  return data_x, data_y

In [4]:
def samplesplitting(data,train,validate,oos,i):
    train_data    = data[(data.index.year >= 1996 )         & (data.index.year < i - validate)]
    validate_data = data[(data.index.year >= i - validate)  & (data.index.year < i)]
    total_train   = data[(data.index.year >= 1996 )         & (data.index.year < i)]
    oos_data      = data[(data.index.year == i)]
    pred_data     = data[(data.index.year >= 1996 + train + validate) & (data.index.year <= i)]

    print('train=%d, validate=%d, oos=%d' % (len(train_data), len(validate_data), len(oos_data)))
    return train_data, validate_data, oos_data, total_train, pred_data

def split_xy(data, kind):
  data_x = data.iloc[:,8:]
  data_y = data[kind]
  return data_x, data_y


# Level old

In [ ]:
# # Complete number of observations is 982285
# # We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# # On average one month contains 3000 options, therefore we loop over 12 * 3000 options, which is similar to looping over a year
# # trainsize = 8 * 3000 * 12
# # validatesize = 5 * 3000 * 12
# # oossize = 3000 * 12

# trainsize = 11 * 2000 * 12
# validatesize = 5 * 2000 * 12
# oossize = 2000 * 12

# # Initialise predictionstring
# IV = 'level'
# oos_pred_level = pd.DataFrame()

# # Hyperparameters
# max_features = [10,15]
# max_depth = [4,6]
# n_estimators = [200]
# n_jobs = -1

# for window in range (0,10):
#   print('Iteration: ', window)
#   best = float("inf")

#   # Optimisation method 1 expanding window

#   # Sample splitting
#   train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
#   train_x, train_y              = split_xy(train,IV)
#   validate_x, validate_y        = split_xy(validate,IV)
#   oos_x, oos_y                  = split_xy(oos,IV)
#   total_train_x, total_train_y  = split_xy(total_train,IV)

#   # Tuning
#   for i in range(len(max_features)):
#     for j in range(len(max_depth)):
#       for k in range(len(n_estimators)):
#         model = RandomForestRegressor(max_features = max_features[i], max_depth=max_depth[j], n_estimators=n_estimators[k], n_jobs = n_jobs)
#         model.fit(train_x, train_y)
#         pred_val= model.predict(validate_x)
#         error = np.sqrt(((pred_val - validate_y) ** 2).mean())
#         if error < best:
#           best = error
#           n_estimators_final = n_estimators[i]
#           max_depth_final = max_depth[j]
#           max_features_final = max_features[k]
#           y_hat = pd.DataFrame(model.predict(oos_x))

#   # Fit model
#   print('Nr of trees: ', n_estimators_final)
#   print('Max Depth: ', max_depth_final)
#   print('Max Features: ', max_features_final)
#   # model_final = RandomForestRegressor(n_estimators=n_estimators_final,
#   #                                    max_depth=max_depth_final,
#   #                                    max_features=max_features_final)
#   # model_final.fit(total_train_x, total_train_y)
#   # y_hat = pd.DataFrame(model_final.predict(oos_x))
#   print('Yearly:',r2_score(oos_y, y_hat))
#   oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True) #oos_pred_level = oos_pred_level.append(y_hat)
#   print('Total:',r2_score(total_oos[IV], oos_pred_level))

#   # Optimisation method 2 GridSearch


# oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/RF/level_predictions_RF.csv', index=True)


Iteration:  0
train=192000, validate=120000, oos=24000
Nr of trees:  200
Max Depth:  4
Max Features:  10
Yearly: 0.33777912401528165
Total: 0.33777912401528165
Iteration:  1
train=216000, validate=120000, oos=24000
Nr of trees:  200
Max Depth:  4
Max Features:  10
Yearly: 0.361828275692759
Total: 0.3505494990659249
Iteration:  2
train=240000, validate=120000, oos=24000
Nr of trees:  200
Max Depth:  4
Max Features:  10
Yearly: 0.39457798407023015
Total: 0.36566706691661854
Iteration:  3
train=264000, validate=120000, oos=24000
Nr of trees:  200
Max Depth:  6
Max Features:  10
Yearly: 0.2594260487291663
Total: 0.3445214657761301
Iteration:  4
train=288000, validate=120000, oos=24000


IndexError: ignored

In [ ]:
# oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/RF/level_predictions_RF.csv', index=True)

# Level new

In [21]:
# Complete number of observations is 982285
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 3000 options, therefore we loop over 12 * 3000 options, which is similar to looping over a year
# trainsize = 8 * 3000 * 12
# validatesize = 5 * 3000 * 12
# oossize = 3000 * 12

trainsize = 11
validatesize = 5
oossize = 1

# Initialise predictionstring
IV = 'level'
oos_pred_level = pd.DataFrame()

# Hyperparameters Best 4, 500, 10 (veel shallow trees)
param_grid = {
    'max_features': [5, 10],#[10, 15],
    'max_depth': [5] ,#[4, 6],
    'n_estimators': [400, 300, 500], #[200, 300], #[200, 500],
    'n_jobs': [-1]
}

# window = 2021
# train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
# total_train_x, total_train_y = split_xy(total_train, IV)
# rf_model = RandomForestRegressor()
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=2, verbose=10)
# grid_search.fit(total_train_x, total_train_y)

# Get the best parameters and create the best model
best_params = {'max_depth': 5, 'max_features': 10, 'n_estimators': 400, 'n_jobs': -1}
# best_params = grid_search.best_params_
print("Best Hyperparameters:", best_params)
best_model = RandomForestRegressor(**best_params)

for window in range(2012, 2021+1):
  print('Iteration: ', window)
  best = float("inf")

  # Sample splitting
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  best_model.fit(total_train_x, total_train_y)
  y_hat = pd.DataFrame(best_model.predict(oos_x))

  # Fit model
  # print('Nr of trees: ', n_estimators_final)
  # print('Max Depth: ', max_depth_final)
  # print('Max Features: ', max_features_final)
  print('Yearly:',r2_score(oos_y, y_hat))
  oos_pred_level = pd.concat([oos_pred_level, y_hat], ignore_index=True)
  print('Total:',r2_score(total_oos[IV], oos_pred_level))

oos_pred_level.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/RF/level_predictions_RF_yearly.csv', index=True)


Best Hyperparameters: {'max_depth': 5, 'max_features': 10, 'n_estimators': 400, 'n_jobs': -1}
Iteration:  2012
train=210839, validate=158189, oos=33864
Yearly: 0.3793025489029699
Total: 0.3793025489029699
Iteration:  2013
train=241135, validate=161757, oos=34182
Yearly: 0.3082563860494091
Total: 0.3510549396390824
Iteration:  2014
train=272284, validate=164790, oos=34113
Yearly: 0.3597969755568664
Total: 0.35501404850158713
Iteration:  2015
train=303150, validate=168037, oos=33074
Yearly: 0.4307937605740929
Total: 0.37519313483793637
Iteration:  2016
train=335237, validate=169024, oos=31906
Yearly: 0.4495714688703072
Total: 0.3923758365783171
Iteration:  2017
train=369028, validate=167139, oos=30972
Yearly: 0.4421602982839664
Total: 0.40198052873955026
Iteration:  2018
train=402892, validate=164247, oos=30002
Yearly: 0.4328076808187883
Total: 0.4061155644092578
Iteration:  2019
train=437074, validate=160067, oos=28538
Yearly: 0.49922740974677504
Total: 0.4167158709180335
Iteration:  20

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.plot(total_oos.index, total_oos['level'] , label='Actual')
plt.plot(total_oos.index, oos_pred_level.values, label='Predicted')
plt.legend()
plt.xlabel('Timestep')
plt.ylabel('y Value')
plt.title('Actual vs Predicted y')
plt.show()

# Slope

In [ ]:
# Complete number of observations is 982285
# We take the initial train, validate and oos sample to be 8, 5 and 13 years respectively (Ratios similar to Gu Kelly Xiu)
# On average one month contains 3000 options, therefore we loop over 12 * 3000 options, which is similar to looping over a year
trainsize = 11
validatesize = 5
oossize = 1

# Initialise predictionstring
IV = 'slope'
oos_pred_slope = pd.DataFrame()

# # Hyperparameters Best
# param_grid = {
#     'max_features': [10,15],#[15, 20, 30],
#     'max_depth': [3,4],#[3,5],
#     'n_estimators': [600, 700],#[500],
#     'n_jobs': [-1]
# }

window = 2021
train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
total_train_x, total_train_y = split_xy(total_train, IV)
# rf_model = RandomForestRegressor()
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=2, verbose=10)
# grid_search.fit(total_train_x, total_train_y)

# # Get the best parameters and create the best model
# best_params = grid_search.best_params_
best_params = {'max_depth': 3, 'max_features': 15, 'n_estimators': 600, 'n_jobs': -1}
print("Best Hyperparameters:", best_params)
best_model = RandomForestRegressor(**best_params)
feature_importance = np.zeros(total_train_x.shape[1])

for window in range(2012, 2021+1):
  print('Iteration: ', window)
  best = float("inf")

  # Sample splitting
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  best_model.fit(total_train_x, total_train_y)

  # Obtain feature importance
  feature_importance += best_model.feature_importances_

  y_hat = pd.DataFrame(best_model.predict(oos_x))
  print('Yearly:',r2_score(oos_y, y_hat))
  oos_pred_slope = pd.concat([oos_pred_slope, y_hat], ignore_index=True)
  print('Total:',r2_score(total_oos[IV], oos_pred_slope))

feature_importance_mean = feature_importance / 10
np.savetxt('/content/drive/MyDrive/Msc Thesis/code/Results/RF/feature_importance_mean_slope_yearly.csv', feature_importance_mean, delimiter=',')
oos_pred_slope.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/RF/slope_predictions_RF_yearly.csv', index=True)

train=504261, validate=148707, oos=27899
Best Hyperparameters: {'max_depth': 3, 'max_features': 15, 'n_estimators': 600, 'n_jobs': -1}
Iteration:  2012
train=210839, validate=158189, oos=33864
Yearly: -0.0196568313264609
Total: -0.0196568313264609
Iteration:  2013
train=241135, validate=161757, oos=34182
Yearly: 0.008869586144049202
Total: -0.0017263955876869463
Iteration:  2014
train=272284, validate=164790, oos=34113
Yearly: -0.03826877799957762
Total: -0.009021025373592728
Iteration:  2015
train=303150, validate=168037, oos=33074
Yearly: -0.048827622069989696
Total: -0.0163337982302747
Iteration:  2016
train=335237, validate=169024, oos=31906
Yearly: -0.016237132962005907
Total: -0.015313263631783425
Iteration:  2017
train=369028, validate=167139, oos=30972
Yearly: 0.021368512712387933
Total: -0.009729633897058276
Iteration:  2018
train=402892, validate=164247, oos=30002


In [ ]:
np.savetxt('/content/drive/MyDrive/Msc Thesis/code/Results/RF/feature_importance_mean_slope.csv', feature_importance_mean, delimiter=',')

# Curvature

In [5]:
trainsize = 11
validatesize = 5
oossize = 1

# Initialise predictionstring
IV = 'curve'
oos_pred_curve = pd.DataFrame()

# # Hyperparameters Best
# param_grid = {
#     'max_features': [20,15], #[10,15],#[15, 20, 30],
#     'max_depth': [5], #[3,4],#[3,5],
#     'n_estimators': [600, 700],#[500],
#     'n_jobs': [-1]
# }

window = 2021
train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
total_train_x, total_train_y = split_xy(total_train, IV)
# rf_model = RandomForestRegressor()
# grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=2, verbose=10)
# grid_search.fit(total_train_x, total_train_y)

# # Get the best parameters and create the best model
# best_params = grid_search.best_params_
best_params = {'max_depth': 5, 'max_features': 20, 'n_estimators': 500, 'n_jobs': -1}
print("Best Hyperparameters:", best_params)
best_model = RandomForestRegressor(**best_params)
feature_importance = np.zeros(total_train_x.shape[1])

for window in range(2012, 2021+1):
  print('Iteration: ', window)
  best = float("inf")

  # Sample splitting
  train, validate, oos, total_train, total_oos = samplesplitting(data_level, trainsize, validatesize, oossize, window)
  train_x, train_y              = split_xy(train,IV)
  validate_x, validate_y        = split_xy(validate,IV)
  oos_x, oos_y                  = split_xy(oos,IV)
  total_train_x, total_train_y  = split_xy(total_train,IV)

  best_model.fit(total_train_x, total_train_y)

  # Obtain feature importance
  feature_importance += best_model.feature_importances_

  y_hat = pd.DataFrame(best_model.predict(oos_x))
  print('Yearly:',r2_score(oos_y, y_hat))
  oos_pred_curve = pd.concat([oos_pred_curve, y_hat], ignore_index=True)
  print('Total:',r2_score(total_oos[IV], oos_pred_curve))

feature_importance_mean = feature_importance / 10
np.savetxt('/content/drive/MyDrive/Msc Thesis/code/Results/RF/feature_importance_mean_curve_yearly.csv', feature_importance_mean, delimiter=',')
oos_pred_curve.to_csv('/content/drive/MyDrive/Msc Thesis/code/Results/RF/curve_predictions_RF_yearly.csv', index=True)

train=504261, validate=148707, oos=27899
Best Hyperparameters: {'max_depth': 5, 'max_features': 20, 'n_estimators': 500, 'n_jobs': -1}
Iteration:  2012
train=210839, validate=158189, oos=33864
Yearly: -0.006536342873224132
Total: -0.006536342873224132
Iteration:  2013
train=241135, validate=161757, oos=34182
Yearly: 0.010351142961254611
Total: 0.004106236401433372
Iteration:  2014
train=272284, validate=164790, oos=34113
Yearly: -0.055894992225641094
Total: -0.012137998100634206
Iteration:  2015
train=303150, validate=168037, oos=33074
Yearly: -0.031278812294906366
Total: -0.013855931594866089
Iteration:  2016
train=335237, validate=169024, oos=31906
Yearly: -0.10347183214519351
Total: -0.02425585276357567
Iteration:  2017
train=369028, validate=167139, oos=30972
Yearly: 0.005597161566876063
Total: -0.02005088662696708
Iteration:  2018
train=402892, validate=164247, oos=30002
Yearly: 0.02878807460337185
Total: -0.01230549062444286
Iteration:  2019
train=437074, validate=160067, oos=285